# Preprocess the authors

This notebook aims to preprocess the author data and save to shelve database,
as well as testing the read write functions

## 1. Import libraries and set up

In [10]:
import shelve
data_cache_prefix = 'data_cache/'

## 2. The process function.

In [14]:
'''
#index 1
#n O. Willum
#a Res. Center for Microperipherik, Technische Univ. Berlin, Germany
#pc 1
#cn 0
#hi 0
#pi 0.0000
#upi 0.0000
#t new product;product group;active product;long product lifetime;old product;product generation;new technology;environmental benefit;environmental choice;environmental consequence
'''
def process_author():
    with open('data/AMiner-Author.txt', 'r') as file:
        data = file.read().split('\n\n')

    with shelve.open('authors.shelve') as author:
        detail = ['n', 'a', 'pc', 'cn', 'hi', 'pi', 'upi', 't']
        for auth in data:
            info = auth.split('\n')
            if info[0] is '':
                break
            index = info[0].split(' ')[1]
            author[index]={}
            for num,each in enumerate(detail):
                author[index][each] = info[num+1].replace('#'+each+' ', '')
            author[index]['a'] = author[index]['a'].split(', ')
            author[index]['t'] = author[index]['t'].split(';')
            author[index]['field'] = []
            author[index]['field']+=([item for field in author[index]['t'] for item in field.split(' ')])
        return author

authors = process_author()

KeyError: 'a'

## 3. Load and insert data

KeyError: 'a'

In [8]:
authors['1']

{'n': 'O. Willum',
 'a': ['Res. Center for Microperipherik',
  'Technische Univ. Berlin',
  'Germany'],
 'pc': '1',
 'cn': '0',
 'hi': '0',
 'pi': '0.0000',
 'upi': '0.0000',
 't': ['new product',
  'product group',
  'active product',
  'long product lifetime',
  'old product',
  'product generation',
  'new technology',
  'environmental benefit',
  'environmental choice',
  'environmental consequence'],
 'field': ['new',
  'product',
  'product',
  'group',
  'active',
  'product',
  'long',
  'product',
  'lifetime',
  'old',
  'product',
  'product',
  'generation',
  'new',
  'technology',
  'environmental',
  'benefit',
  'environmental',
  'choice',
  'environmental',
  'consequence']}

In [ ]:
def insert_author(conn, author):
    cur = conn.cursor()
    sql = '''
        INSERT INTO authors(
    '''
